# Example to display a IW L1B x-spectra

## read L1B data

In [ ]:
import glob
import os
import datatree
from slcl1butils.utils import get_test_file
#l1bncfile_pattern = os.path.abspath('../../assests/*iw*nc')
one_safe_l1b = get_test_file('S1B_IW_XSP__1SDV_20211026T045709_20211026T045736_029302_037F35_1CD7.SAFE')
l1bncfile_pattern = os.path.join(one_safe_l1b,'*iw*vv*nc')
print(l1bncfile_pattern)
lst = glob.glob(l1bncfile_pattern)
l1bncfile = lst[0]
l1bncfile

In [ ]:
dt = datatree.open_datatree(l1bncfile)
dt

## pick up intra or inter burst group

In [ ]:
#cat_xspec = 'inter'
cat_xspec = 'intra'
ds = dt[cat_xspec+'burst'].to_dataset()
ds

## recombine real part and imaginary part (as recommanded by Fred)

In [ ]:
for tautau in range(3):
    ds['xspectra_%stau'%tautau] = ds['xspectra_%stau_Re'%tautau] + 1j*ds['xspectra_%stau_Im'%tautau]
    ds = ds.drop(['xspectra_%stau_Re'%tautau,'xspectra_%stau_Im'%tautau])
ds

# get corners from the tiles in each bursts of the sub-swath

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from shapely.geometry import Polygon
all_poly = {}
all_poly_coords = {}
for bursti in range(ds.burst.size):
    for i_tile_sample in range(ds.tile_sample.size):
        for i_tile_line in range(ds.tile_line.size):
            #print('bursti',bursti,i_tile_sample,i_tile_line)
            lonco  = ds['corner_longitude'].isel({'burst':bursti,'tile_sample':i_tile_sample,'tile_line':i_tile_line}).values.ravel(order='C')
            latco  = ds['corner_latitude'].isel({'burst':bursti,'tile_sample':i_tile_sample,'tile_line':i_tile_line}).values.ravel(order='C')
            coords = np.stack([lonco,latco]).T
            #print('coords',coords)
            coords = coords[[0,1,3,2,0]]
            #print('coords2',coords)
            pp = Polygon(coords)
            #all_poly.append(pp)
            all_poly['burst%s_sample%s_line%s'%(bursti,i_tile_sample,i_tile_line)] = pp
            all_poly_coords['burst%s_sample%s_line%s'%(bursti,i_tile_sample,i_tile_line)] = coords
print('done',len(all_poly.keys()))

In [ ]:
import geoviews as gv
import holoviews as hv
gv.extension('bokeh')
from shapely.geometry import Polygon
import shapely.wkt

P = shapely.wkt.loads(ds.footprint)
#poly = Polygon(np.vstack([lons_tiles,lats_tiles]).T)
print(P)
foot = gv.Path(P).opts(color='magenta')
tmp =[]
for uu in  all_poly:
    tmp.append(gv.Path(all_poly[uu]))
(gv.tile_sources.Wikipedia * foot * hv.Overlay(tmp) ).opts(width=800,height=600,title='%s tiles SAR Sentinel-1 L1B IFREMER '%(cat_xspec))

# display pyplot figure of cross spectrum

In [ ]:


import slcl1butils.plotting.display_one_spectra
from importlib import reload
reload(slcl1butils.plotting.display_one_spectra)

tile_line_i = 0
tile_sample_i = 3
fig = plt.figure()
slcl1butils.plotting.display_one_spectra.plot_a_single_xspec_cart_L1B_IW(ds,bursti,tile_line_i,tile_sample_i,'test display',fig,cat_xspec=cat_xspec,part='Re')